In [12]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from dotenv import load_dotenv
import openai
from openai import OpenAI

In [13]:
load_dotenv()

True

In [14]:
client = OpenAI(api_key=os.getenv('OPEN_API_KEY'))

### Data Ingestion

In [8]:

# # Open the PDF file
# pdf_path = "Home_Insurance_Policy.pdf"
# doc = fitz.open(pdf_path)

# # Extract text from all pages
# documents = []
# for page_num in range(doc.page_count):
#     page = doc.load_page(page_num)  # Load each page
#     text = page.get_text()  # Extract text from the page
#     documents.append(text)

In [15]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
    return text

# Example usage
pdf_path = "Home_Insurance_Policy.pdf"
pdf_text = extract_text_from_pdf(pdf_path)


In [16]:
from minsearch import MinSearch

# Initialize MinSearch
search_index = MinSearch()

# Split the PDF text into smaller chunks for indexing
# You can decide the best way to chunk your text (e.g., by sentences, paragraphs, or a custom logic)
chunks = pdf_text.split('\n\n')  # This example splits by paragraphs

# Index each chunk of text with an identifier (e.g., the paragraph number)
for i, chunk in enumerate(chunks):
    search_index.add(i, chunk)  # `i` is the ID, `chunk` is the content

# Commit the index
search_index.commit()

ImportError: cannot import name 'MinSearch' from 'minsearch' (/Users/monahaghighi/.local/share/virtualenvs/llm_final_project-WhKreUth/lib/python3.9/site-packages/minsearch/__init__.py)

In [17]:
index = minsearch.Index(
    text_fields= documents
)


NameError: name 'minsearch' is not defined

In [18]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [19]:
prompt_template = """
You're an Insurance agent. Answer the QUESTION based on the CONTEXT from property insurance document.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()


def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [20]:
def rag(query, model='gpt-4o-mini'):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    #print(prompt)
    answer = llm(prompt, model=model)
    return answer

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Ensure that you process each document individually and then combine the chunks
chunks = []
for doc in documents:
    chunks.extend(text_splitter.split_text(doc))  # Split each document

NameError: name 'documents' is not defined

In [51]:
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

In [54]:
response = embeddings.embed_documents(["This is a test document."])
print(response)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ful************-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [49]:
vector_store = FAISS.from_texts(chunks, embeddings)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-ful************-key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [34]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_text(documents)

# Step 4: Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))

# Step 5: Create FAISS vector store from document chunks
vector_store = FAISS.from_texts(chunks, embeddings)

# Step 6: Set up the retriever (optional step based on what you're doing next)
retriever = vector_store.as_retriever()

# Step 7: Use the retriever to query your FAISS index (optional example query)
query = "Your query here"
docs = retriever.get_relevant_documents(query)

# Display the retrieved documents
for doc in docs:
    print(doc)

TypeError: expected string or bytes-like object

In [16]:
def minsearch_search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [17]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [55]:
from langchain.retrievers import ElasticsearchRetriever

# Define your Elasticsearch endpoint here
es_endpoint = "http://localhost:9200"
index_name = "insurance-docs"

# Elasticsearch Retriever setup
retriever = ElasticsearchRetriever(
    elasticsearch_url=es_endpoint,
    index_name=index_name
)

ImportError: cannot import name 'ElasticsearchRetriever' from 'langchain.retrievers' (/Users/monahaghighi/.local/share/virtualenvs/llm_final_project-WhKreUth/lib/python3.9/site-packages/langchain/retrievers/__init__.py)